In [ ]:
from notebooks import *

In [ ]:
load = Load()
recs = recs if 'recs' in locals() else load.recs(
    # limit=30,  # XXX Faster dev
    datasets=[
        'peterson-field-guide',
        # 'recordings',
    ],
)
display(
    # df_summary(recs).T,
    # df_summary(recs),
    # recs[:5],
)

.

In [ ]:
# (recs
#     .assign(species=lambda df: df.species_longhand)
#     .assign(count=1)
#     # 0-fill all (species, dataset) combinations to create empty placeholders for missing bars
#     .pipe(lambda df: df.append(
#         pd.DataFrame([
#             dict(species=species, dataset=dataset, duration_s=0)
#             for species in df.species.unique()
#             for dataset in df.dataset.unique()
#         ])
#         .astype({'species': df.species.dtype})
#     ))
#     .groupby(['dataset', 'species'])[['count', 'duration_s']].sum().reset_index()
#     .pipe(pd.melt, id_vars=['dataset', 'species'], value_vars=['count', 'duration_s'])
#     .pipe(df_reverse_cat, 'species')
#     .pipe(ggplot, aes(x='species', y='value', fill='dataset', color='dataset'))
#     + coord_flip()
#     + geom_bar(stat='identity', position=position_dodge(), width=.8)
#     + facet_wrap('variable', nrow=1, scales='free')
#     + xlab('')
#     + ylab('')
#     + scale_fill_cmap_d(mpl.cm.tab10)
#     + scale_color_cmap_d(mpl.cm.tab10)
#     + theme(panel_spacing=2.5)
#     + theme_figsize(width=10, aspect_ratio=3/1)
#     + ggtitle(f'recs: Total (count, duration_s) per (species, dataset)')
# )

In [ ]:
# Load a saved projection model
projection = Projection.load('peterson-v0-26bae1c', features=Features(load=load))

[18:31:27.241] Projection.load
  path: /Users/danb/hack/bubo/data/models/projection/peterson-v0-26bae1c.pkl


In [ ]:
# Add .feat
recs = projection.transform(recs)

.

[                                                                                          ] | 0% Completed |  0.0s

.

.

.

...

.

..

[#####################################################################################     ] | 94% Completed |  1.2s

.

[##########################################################################################] | 100% Completed |  1.3s

In [ ]:
%%prun -l20
# GridSearchCV many models / model params
params = dict(
    classifier = [

        # # 'cls: knn, n_neighbors: 1',
        # # 'cls: knn, n_neighbors: 3',
        # 'cls: knn, n_neighbors: 5',
        # 'cls: knn, n_neighbors: 10',
        # # 'cls: knn, n_neighbors: 15',
        # 'cls: knn, n_neighbors: 25',
        # 'cls: knn, n_neighbors: 50',

        # 'cls: svm, random_state: 0, probability: true, kernel: rbf, C: 1',
        # 'cls: svm, random_state: 0, probability: true, kernel: rbf, C: 10',  # Like [SBF16]
        # 'cls: svm, random_state: 0, probability: true, kernel: rbf, C: 100',  # Same as C: 10 [why?]
        # # 'cls: svm, random_state: 0, probability: true, kernel: rbf, C: 1000',  # Same as C: 10 [why?]

        # # 'cls: rf, random_state: 0, criterion: gini, n_estimators: 10',  # Defaults
        # 'cls: rf, random_state: 0, criterion: gini, n_estimators: 50',
        # 'cls: rf, random_state: 0, criterion: gini, n_estimators: 100',
        # 'cls: rf, random_state: 0, criterion: gini, n_estimators: 200',
        # 'cls: rf, random_state: 0, criterion: gini, n_estimators: 400',

        # 'cls: rf, random_state: 0, criterion: entropy, n_estimators: 10',
        'cls: rf, random_state: 0, criterion: entropy, n_estimators: 50',
        'cls: rf, random_state: 0, criterion: entropy, n_estimators: 100',
        'cls: rf, random_state: 0, criterion: entropy, n_estimators: 200',  # Like [SP14]
        'cls: rf, random_state: 0, criterion: entropy, n_estimators: 400',

    ]
    # TODO agg_funs
)
select = sk.model_selection.GridSearchCV(
    Search(projection=projection),
    params,
    # cv=3,  # [SP14] uses two-/three-fold CV [why?]
    cv=sk.model_selection.ShuffleSplit(
        # n_splits=1,
        n_splits=5,
        # n_splits=10,
        # n_splits=20,
        test_size=.2,
        random_state=0,
    ),
    return_train_score=True,
    # verbose=100,
    # verbose=10,  # O(models * n_splits) lines of outputs
    verbose=1,  # O(1) lines of outputs
    n_jobs=1,  # For %prun
    # n_jobs=8,
    # n_jobs=16,
)
with contextlib.ExitStack() as stack:
    # stack.enter_context(cache_control(refresh=True))
    # stack.enter_context(joblib.parallel_backend('threading'))  # Default: 'multiprocessing'
    # stack.enter_context(joblib.parallel_backend('sequential'))  # For %prun [FIXME Has no effect?]
    stack.enter_context(log.context(verbose=False))
    X, y = Search.Xy(recs)
    select.fit(X, y)

In [ ]:
select.cv_results_

{
  'mean_fit_time': array([0.046, 0.079, 0.135, 0.265]),
  'mean_score_time': array([0.053, 0.058, 0.065, 0.089]),
  'mean_test_score': array([-4.2, -3.6, -3. , -3.2]),
  'mean_train_score': array([-1., -1., -1., -1.]),
  'param_classifier': masked_array(data=['cls: rf, random_state: 0, criterion: entropy, n_estimators: 50',
                     'cls: rf, random_state: 0, criterion: entropy, n_estimators: 100',
                     'cls: rf, random_state: 0, criterion: entropy, n_estimators: 200',
                     'cls: rf, random_state: 0, criterion: entropy, n_estimators: 400'],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [
    {'classifier': 'cls: rf, random_state: 0, criterion: entropy, n_estimators: 50'},
    {'classifier': 'cls: rf, random_state: 0, criterion: entropy, n_estimators: 100'},
    {'classifier': 'cls: rf, random_state: 0, criterion: entropy, n_estimators: 200'},
    {'classifier': 'cls: rf, 

In [ ]:
# CV runtimes
# with warnings.catch_warnings():
#     # TODO How to avoid the 'color' legend warning since geom_col has no color mapping?
#     warnings.simplefilter(action='ignore', category=UserWarning)
#     repr(cv_results_summary_df(select.cv_results_)
#         .filter(regex='^(param_.*|.*_time$)')
#         .pipe(df_flatmap, lambda row: [
#             row.set_value('stage', 'fit').rename({'mean_fit_time': 'mean_time', 'std_fit_time': 'std_time'}),
#             row.set_value('stage', 'score').rename({'mean_score_time': 'mean_time', 'std_score_time': 'std_time'}),
#         ])
#         .filter(regex='^(param.*|stage|mean_time|std_time)$')
#         .pipe(df_ordered_cat, 'param_classifier', transform=reversed)
#         .pipe(ggplot, aes(x='param_classifier'))
#         + geom_col(aes(y='mean_time', fill='stage'), position=position_dodge())
#         + geom_linerange(
#             aes(group='stage', y='mean_time', ymin='mean_time - 1.96 * std_time', ymax='mean_time + 1.96 * std_time'),
#             position=position_dodge(width=.9),
#         )
#         + coord_flip()
#         + scale_color_cmap_d(mpl.cm.Set1)
#         # + theme(legend_position='bottom')
#         + theme_figsize(width=4, aspect_ratio=2/1)
#         + ggtitle('Train/test runtimes (WARNING: mix of cached and uncached)')
#     )

In [ ]:
(cv_results_splits_df(select.cv_results_)
    .assign(
        classifier_config=lambda df: df.param_classifier.apply(lambda x: yaml.safe_load('{%s}' % x)),
        type=lambda df: df.classifier_config.apply(lambda x: (
            '%s-%s' % (x['cls'], x['criterion']) if x['cls'] == 'rf' else
            x['cls']
        )),
        group=lambda df: df.apply(axis=1, func=lambda row: row.type + '/' + row.split),
    )
    .pipe(df_ordered_cat, param_classifier=lambda df: params['classifier'])
    .pipe(ggplot, aes(x='param_classifier', y='score', color='group'))
    + geom_hline(yintercept=0, color='grey')
    + geom_jitter(alpha=.5, width=.05)
    + geom_line(aes(group='group + fold'), alpha=.2)
    + stat_summary(aes(group='group'), fun_data='mean_cl_boot', random_state=0, geom='errorbar')
    + stat_summary(aes(group='group'), fun_data='mean_cl_boot', random_state=0, geom='line', size=1)
    + scale_color_cmap_d(mpl.cm.tab10)
    + theme(axis_text_x=element_text(angle=90, hjust=.5))
    + theme_figsize(width=12, aspect_ratio=1/2)
    + ylab('score (-coverage_error)')
    + ggtitle('Train/test scores')
)

In [ ]:
cv_splits = list(select.cv.split(range(len(recs))))
with log.context(verbose=False):
    models = (
        DF([
            OrderedDict(
                model_id='%s; fold: %s' % (param_classifier, fold),
                param_classifier=param_classifier,
                fold=fold,
                train_i=train_i,
                test_i=test_i,
            )
            for param_classifier in params['classifier']
            for fold, (train_i, test_i) in enumerate(cv_splits
                # [:2]  # XXX dev
            )
            for X, y in [Search.Xy(recs)]
        ])
        # [:3]  # XXX dev
        .pipe(
            df_apply_with_progress,
            # use='sync',  # [~30s]
            use='dask', scheduler='threads', npartitions=16,  # [~20-30s]
            # use='joblib', backend='threading', n_jobs=16, verbose=10,  # [~20-30s]
            # use='joblib', backend='threading', n_jobs=16, verbose=100,
            f=lambda row: row.append(S(dict(
                search=(
                    sk.base.clone(select.estimator)
                    .set_params(classifier=row.param_classifier)
                    .fit(
                        Search.X(recs)[row.train_i],
                        Search.y(recs)[row.train_i],
                    )
                ),
            ))),
        )
    )

display(
    len(params['classifier']),
    len(cv_splits),
    models.shape,
    models[:1].T,
)

# Model diagnostics: all models, all folds

In [ ]:
# print("params['classifier']:\n%s" % '\n'.join('  %s: %r' % (i, x) for i, x in enumerate(params['classifier'])))
# print()
print('models:')
print('  param_classifier[*/%s]' % len(params['classifier']))
print('  fold[*/%s]' % select.cv.n_splits)

In [ ]:
# TODO This is slow (~2m for 400 its) without par (but par makes it a slog to write...)
coverage_errors_all_all = (models
    # .sample(n=5, random_state=0)  # For faster dev
    .pipe(lambda df: DF(
        OrderedDict(
            **row[['model_id', 'param_classifier', 'fold']],
            **out_row[['species', 'coverage_error', 'id']],
        )
        for row in iter_progress(df_rows(df), n=len(df))
        for out_row in df_rows(row.search.coverage_errors_recs(recs.iloc[row.test_i]))
    ))
)
display(
    coverage_errors_all_all[:5],
)

In [ ]:
%%time
_coverage_errors_all_all_precompute = (coverage_errors_all_all
    # [:5]
    .assign(
        # TODO Push this upstream, very slow (probably the yaml.safe_load)
        classifier_config=lambda df: df.param_classifier.apply(lambda x: yaml.safe_load('{%s}' % x)),
        type=lambda df: df.classifier_config.apply(lambda x: (
            '%s-%s' % (x['cls'], x['criterion']) if x['cls'] == 'rf' else
            x['cls']
        )),
    )
)

In [ ]:
# TODO TODO
# in: coverage_errors_all_all, recs
(_coverage_errors_all_all_precompute
    [lambda df: df.param_classifier.isin([
        # 'cls: knn, n_neighbors: 1',
        # 'cls: knn, n_neighbors: 3',
        'cls: knn, n_neighbors: 5',
        'cls: knn, n_neighbors: 10',
        'cls: knn, n_neighbors: 15',
        'cls: knn, n_neighbors: 50',
        'cls: svm, random_state: 0, probability: true, kernel: rbf, C: 1',
        'cls: svm, random_state: 0, probability: true, kernel: rbf, C: 10',
        'cls: svm, random_state: 0, probability: true, kernel: rbf, C: 100',
        # 'cls: svm, random_state: 0, probability: true, kernel: rbf, C: 1000',
        # 'cls: rf, random_state: 0, criterion: gini, n_estimators: 10',
        'cls: rf, random_state: 0, criterion: gini, n_estimators: 50',
        'cls: rf, random_state: 0, criterion: gini, n_estimators: 100',
        'cls: rf, random_state: 0, criterion: gini, n_estimators: 200',
        'cls: rf, random_state: 0, criterion: gini, n_estimators: 400',
        # 'cls: rf, random_state: 0, criterion: entropy, n_estimators: 10',
        'cls: rf, random_state: 0, criterion: entropy, n_estimators: 50',
        'cls: rf, random_state: 0, criterion: entropy, n_estimators: 100',
        'cls: rf, random_state: 0, criterion: entropy, n_estimators: 200',
        'cls: rf, random_state: 0, criterion: entropy, n_estimators: 400',
    ])]
    # Sort species by median(coverage_error) [across all models...]
    .pipe(df_ordered_cat, species=lambda df: (
        df.groupby('species').agg({'coverage_error': np.median}).reset_index().sort_values('coverage_error').species
    ))
    .pipe(ggplot, aes(x='species', y='coverage_error', color='param_classifier'))
    # TODO TODO Facet by species! -- to see model x species variation
    # + stat_summary(aes(group='param_classifier'), fun_y=np.median, geom='line')
    + facet_wrap('type') + stat_summary(aes(group='param_classifier'), fun_y=np.median, geom='line')
    # + facet_wrap('species')  # No geom_line here  [TODO species is already mapped to x, that's why this is broken (duh)]
    # TODO Also increase cv to decrease variance in eval metrics (this made Alex extremely squirmy)
    + coord_flip()
    # + geom_hline(yintercept=recs.species.nunique(), color='grey')
    + scale_color_cmap_d(mpl.cm.tab20)
    + theme_figsize('square')
    + ggtitle(rf'Coverage error for fold $\times$ instance, by param_classifier')
)

# Model diagnostics: one model, all folds

In [ ]:
param_classifier_i = 19
param_classifier = params['classifier'][param_classifier_i]
print("params['classifier']:\n%s" % '\n'.join('  %s: %r' % (i, x) for i, x in enumerate(params['classifier'])))
print()
print('models:')
print('  param_classifier[%s/%s]: %r' % (param_classifier_i, len(params['classifier']), param_classifier))
print('  fold[*/%s]' % select.cv.n_splits)

In [ ]:
coverage_errors_one_all = (models
    [lambda df: df.param_classifier == param_classifier]  # One model, all folds
    # [:2]  # For faster dev
    .pipe(lambda df: DF(
        OrderedDict(
            **row[['model_id', 'param_classifier', 'fold']],
            **out_row[['species', 'coverage_error', 'id']],
        )
        for row in iter_progress(df_rows(df), n=len(df))
        for out_row in df_rows(row.search.coverage_errors_recs(recs.iloc[row.test_i]))
    ))
)
display(
    coverage_errors_one_all[:5],
)

In [ ]:
# in: coverage_errors_one_all, recs
(coverage_errors_one_all
    # Sort species by median(coverage_error)
    .pipe(df_ordered_cat, species=lambda df: (
        df.groupby('species').agg({'coverage_error': np.median}).reset_index().sort_values('coverage_error').species
    ))
    .pipe(ggplot, aes(x='species', y='coverage_error'))
    + geom_count(aes(size='..n..'))  # n (count) instead of prop (proportion)
    + scale_size_area()  # Instead of default scale_size(), because it's a count [I don't grok this but it looks good]
    + stat_summary(fun_y=np.median, geom='point', color='red', alpha=.5, shape='|', size=6, stroke=1)
    + stat_summary(
        fun_ymin=partial(np.percentile, q=25), fun_ymax=partial(np.percentile, q=75), geom='linerange', color='red', alpha=.5, size=1,
    )
    + coord_flip()
    + geom_hline(yintercept=recs.species.nunique(), color='grey')
    + theme_figsize('square')
    + ggtitle(rf'Coverage error for fold $\times$ instance ({param_classifier})')
)

In [ ]:
(coverage_errors_one_all
    [:5]
)

In [ ]:
with figsize('full'):
    search.plot_confusion_matrix(test_recs, title=model_id)

# Model diagnostics: one model, one fold

In [ ]:
fold = 11
# param_classifier_i = ...  # Comment out to reuse from above
param_classifier = params['classifier'][param_classifier_i]
[(_, model)] = list(models[lambda df: (df.param_classifier == param_classifier) & (df.fold == fold)].iterrows())
print("params['classifier']:\n%s" % '\n'.join('  %s: %r' % (i, x) for i, x in enumerate(params['classifier'])))
print()
print('model:')
print('  param_classifier[%s/%s]: %r' % (param_classifier_i, len(params['classifier']), model.param_classifier))
print('  fold[%s/%s]' % (model.fold, select.cv.n_splits))
print()
print(model)

In [ ]:
# in: model
model_id = model.model_id
param_classifier = model.param_classifier
fold = model.fold
train_i = model.train_i
test_i = model.test_i
search = model.search

# in: model, recs
train_recs = recs.iloc[model.train_i]
train_X = Search.X(recs)[model.train_i]
train_y = Search.y(recs)[model.train_i]
test_recs = recs.iloc[model.test_i]
test_X = Search.X(recs)[model.test_i]
test_y = Search.y(recs)[model.test_i]

display(
    len(train_recs),
    len(test_recs),
)

In [ ]:
(search.coverage_errors_recs_by(test_recs, 'id')
    .merge(test_recs[['id', 'species']], on='id', how='left')
    .pipe(ggplot, aes(x='species', y='coverage_error'))
    + geom_count(aes(size='..n..'))
    + stat_summary(fun_y=np.median, geom='point', color='red', alpha=.5, shape='|', size=6, stroke=1)
    + stat_summary(
        fun_ymin=partial(np.percentile, q=25), fun_ymax=partial(np.percentile, q=75), geom='linerange', color='red', alpha=.5, size=1,
    )
    + coord_flip()
    + geom_hline(yintercept=len(search.classes_), color='grey')
    + scale_x_discrete(limits=list(reversed(test_recs.species.cat.categories)))
    + theme_figsize('square')
    + ggtitle(f'Coverage error by species ({model_id})')
)

In [ ]:
search.confusion_matrix(test_recs)

In [ ]:
# TODO TODO
_search, _recs = search, test_recs
species = _search.species(_recs)
y_true = species.species_true
y_pred = species.species_pred
labels = sorted_species(set(y_true) | set(y_pred))
pd.DataFrame(
    data=sk.metrics.confusion_matrix(y_true, y_pred, labels),
    index=pd.Series(labels, name='true'),
    columns=pd.Series(labels, name='pred'),
)

In [ ]:
with figsize('full'):
    search.plot_confusion_matrix(test_recs, title=model_id)